In [ ]:
%load_ext autoreload
%autoreload 2

import sys

In [ ]:
import os
import array
import re

import pandas
import numpy

import plotly.express
import librosa
from matplotlib import pyplot as plt

from software.features.spectral import design_bandpass, plot_bandpass
from software.features.featureutils import resample, rms, vector_magnitude
from software.dataset.combine import load_data, read_labels, parse_video_filename

In [ ]:
sensor_data_path = 'data/jonnor-brushing-1/har_record/'
labels_path  = 'data/jonnor-brushing-1/labels/project-7-at-2024-12-31-23-50-84589958.csv'

samplerate = 50
lower = 2.0
upper = 5.0

In [ ]:
bandpass_sos = design_bandpass(lower=lower, upper=upper, sr=samplerate, order=2)
#plot_bandpass(bandpass_sos, lower=lower, upper=upper, sr=samplerate);

In [ ]:
# Load sensor data
data = load_data(sensor_data_path)
print(data.columns)
print(data.head(5))

In [ ]:


# Load labels
labels = read_labels(labels_path)
#labels['start_time'] = labels['']
print(labels.filename.unique())


In [ ]:
labels['gdrive_id'] = labels['filename'].str.replace('ucexportdownloadid', '')
labels.gdrive_id.unique()

In [ ]:
videos_path = 'data/jonnor-brushing-1/videos.csv'
videos = pandas.read_csv(videos_path)
videos['file_time'] = videos.filename.apply(parse_video_filename)
gdrive_prefix = 'https://drive.google.com/uc?export=download&id='
videos['gdrive_id'] = videos['data_url'].str.replace(gdrive_prefix, '')
videos

In [ ]:
mm = pandas.merge(labels, videos, left_on='gdrive_id', right_on='gdrive_id')
mm['start_time'] = mm['file_time'] + pandas.to_timedelta(mm['start'], unit='s')
mm['end_time'] = mm['file_time'] + pandas.to_timedelta(mm['end'], unit='s')
mm

In [ ]:

mm = pp.reset_index()
#mm = pandas.merge(mm, filename_meta, left_on='filename', right_on='filename')
mm['motion_mag'] = vector_magnitude(mm[['motion_x', 'motion_y', 'motion_z']], axis=1)
mm['bandpass_motion_mag'] = vector_magnitude(mm[['bandpass_motion_x', 'bandpass_motion_y', 'bandpass_motion_z']], axis=1)
mm

In [ ]:


freq = '200ms'

re1 = resample(sub[['bandpass_motion_mag']], freq=freq, func=lambda a, **kwargs: rms(a.values, axis=0))
re2 = resample(sub[['norm_gravity_pca0']], freq=freq, func='mean')

m = pandas.merge(re1, re2, right_index=True, left_index=True)
m = pandas.merge(m, sub, right_index=True, left_index=True)
m

In [ ]:
fig = plotly.express.line(m.reset_index(),
                          x='elapsed',
                          y=['norm_gravity_pca0_x', 'bandpass_motion_mag_x'],
                          #y=['pca0', 'pca1'],
                          facet_row='filename',
                          facet_row_spacing=0.01,
                          width=2000,
                          height=200*42)
fig.update_layout(showlegend=False)

In [ ]:
print(m.elapsed.max())